## Spark Structured Streaming for Incremental Load (File-based Ingestion)

### Scenario
A retail company receives daily sales transaction files from multiple store locations into a cloud folder (e.g., Azure Data Lake / Unity Catalog Volume).

Instead of reprocessing all historical files every day, we use **Spark Structured Streaming** to:
- Automatically detect **newly arrived files**
- Ingest them incrementally into a **Delta table**
- Keep analytics tables/dashboards up-to-date while reducing compute and processing time

### What we are building
Source (landing folder with new CSV files) ➜ Structured Streaming ➜ Delta sink path (data)  
+ a checkpoint directory to track progress (exactly-once semantics)

### Paths used in this example

**Source (new files arrive here):**
`/Volumes/pyspark_cata/source/db_volume/spark_stream/`

**Delta sink (output table stored as files):**
`/Volumes/pyspark_cata/source/db_volume/stream_sink/data/`

**Checkpoint (stream state & progress):**
`/Volumes/pyspark_cata/source/db_volume/stream_sink/checkpoint/`

> The checkpoint is mandatory for reliable streaming. It stores offsets and metadata so we don’t reprocess the same files again.

### Define a consistent schema (recommended for CSV streams)

For file-based streaming, Spark needs a stable schema to parse incoming CSV files.
Defining it explicitly helps avoid inference issues and type mismatches as new files arrive.

In [0]:
stream_schema = """
    order_id INT,
    customer_id INT,
    order_date DATE,
    amount DOUBLE
"""

### Quick batch read (sanity check)

Before starting the stream, we do a one-time batch read from the same source folder to:
- confirm schema correctness
- confirm the folder path and file format
- preview the data

In [0]:
df_batch = spark.read.format("csv")\
    .option("header", True)\
    .schema(stream_schema)\
    .load("/Volumes/pyspark_cata/source/db_volume/spark_stream/")
# Batch read = quick validation step (schema + path + preview)
display(df_batch)

order_id,customer_id,order_date,amount
1,101,2025-08-02,246.84
2,104,2025-08-03,111.3
3,103,2025-08-04,52.0
4,103,2025-08-05,98.7
5,102,2025-08-06,392.67


### Start streaming read from the landing folder

Structured Streaming will treat new files arriving in the folder as a continuous stream.
Each micro-batch processes only files that have not been processed yet (tracked by checkpoint).

In [0]:
df = spark.readStream.format("csv")\
    .option("header", True)\
    .schema(stream_schema)\
    .load("/Volumes/pyspark_cata/source/db_volume/spark_stream/")
# Streaming read = continuously watches the folder for newly arrived files

### Write the stream into a Delta sink

We write streaming output to a Delta path and configure:
- **checkpointLocation**: stream progress/state (required)
- **mergeSchema**: allow schema evolution if new columns appear (use carefully)
- **trigger(once=True)**: runs one micro-batch and stops (good for scheduled incremental loads)

> `trigger(once=True)` makes streaming behave like an incremental batch job.
> If you want continuous ingestion, you would remove this and use a processing-time trigger.

In [0]:
df.writeStream.format("delta")\
    .option("checkpointLocation", "/Volumes/pyspark_cata/source/db_volume/stream_sink/checkpoint")\
    .option("mergeSchema", "true")\
    .trigger(once=True)\
    .start("/Volumes/pyspark_cata/source/db_volume/stream_sink/data")
# checkpointLocation tracks processed files (prevents duplicates on reruns)
# trigger(once=True) makes this run like an incremental batch job (process new files, then stop)

### Verify the Delta sink

We query the Delta path directly to confirm that the stream wrote records successfully.

In [0]:
%sql
SELECT * FROM delta.`/Volumes/pyspark_cata/source/db_volume/stream_sink/data/`

order_id,customer_id,order_date,amount
1,101,2025-08-02,246.84
2,104,2025-08-03,111.3
3,103,2025-08-04,52.0
4,103,2025-08-05,98.7
5,102,2025-08-06,392.67


### What happens on rerun?

If you run the stream again with the same checkpoint:
- Spark will **not reprocess** already ingested files
- Only **new files** added to the source folder will be picked up

This is the main benefit of using checkpointing for file-based incremental ingestion.

In [0]:
df.writeStream.format("delta")\
    .option("checkpointLocation", "/Volumes/pyspark_cata/source/db_volume/stream_sink/checkpoint")\
        .option("mergeSchema", "true")\
            .trigger(once=True)\
                .start("/Volumes/pyspark_cata/source/db_volume/stream_sink/data")

In [0]:
%sql
SELECT * FROM delta.`/Volumes/pyspark_cata/source/db_volume/stream_sink/data/`

order_id,customer_id,order_date,amount
6,100,2025-08-07,248.69
7,102,2025-08-08,243.85
8,101,2025-08-09,308.31
9,105,2025-08-10,367.45
10,105,2025-08-11,328.2
1,101,2025-08-02,246.84
2,104,2025-08-03,111.3
3,103,2025-08-04,52.0
4,103,2025-08-05,98.7
5,102,2025-08-06,392.67
